https://adventofcode.com/2023/day/20

In [1]:
import math
from collections import defaultdict, deque, Counter
from itertools import pairwise

In [2]:
with open("data/20.txt") as f:
    data = f.read()

In [3]:
testdata = """\
broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output"""

In [4]:
class Broadcaster:
    def __init__(self, modules, label, targetstr=""):
        self.label = label
        self.modules = modules
        self.targets = set(targetstr.split(", "))
        self.pulses = defaultdict(int)

    def add_targets(self, targetstr):
        self.targets.update(targetstr.split(", "))

    def receive(self, pulse):
        def callback():
            self.pulses[pulse] += 1
            for t in self.targets:
                yield self.modules[t].receive(self.label, pulse)

        callback.label = self.label
        callback.source = "none"
        callback.pulse = pulse
        return callback


class Output:
    def __init__(self):
        self.pulses = defaultdict(int)

    def receive(self, _, pulse):
        self.pulses[pulse] += 1


class FlipFlop:
    def __init__(self, modules, label, targetstr=""):
        self.label = label
        self.modules = modules
        self.targets = set(targetstr.split(", "))
        self.val = 0
        self.pulses = defaultdict(int)

    def add_targets(self, targetstr):
        self.targets.update(targetstr.split(", "))

    def receive(self, source, pulse):
        def callback():
            self.pulses[pulse] += 1
            if not pulse:
                self.val = 1 - self.val
                for t in self.targets:
                    yield self.modules[t].receive(self.label, self.val)

        callback.label = self.label
        callback.source = source
        callback.pulse = pulse
        return callback

    def __repr__(self):
        return f"< FlipFlop {self.label} >"


class Conjunction:
    def __init__(self, modules, label, targetstr=""):
        self.label = label
        self.modules = modules
        self.targets = set(targetstr.split(", "))
        self.inputs = {}
        self.pulses = defaultdict(int)

    def add_targets(self, targetstr):
        self.targets.update(targetstr.split(", "))

    def add_input(self, input):
        self.inputs[input] = 0

    def receive(self, source, pulse):
        def callback():
            self.pulses[pulse] += 1
            self.inputs[source] = pulse
            if not self.inputs:
                return
            if all(x == 1 for x in self.inputs.values()):
                out = 0
            else:
                out = 1
            for t in self.targets:
                yield modules[t].receive(self.label, out)

        callback.label = self.label
        callback.source = source
        callback.pulse = pulse
        return callback

    def __repr__(self):
        return f"< Conjunction {self.label} >"


def parse_puzzle(puzzle):
    modules = defaultdict(Output)
    pending_targets = defaultdict(list)
    conjunctions = set()
    for line in puzzle.splitlines():
        modulestr, targetstr = line.split(" -> ")
        if modulestr == "broadcaster":
            modules["broadcaster"] = Broadcaster(modules, "broadcaster", targetstr)
            continue
        mtype, label = modulestr[0], modulestr[1:]
        if mtype == "%":
            modules[label] = FlipFlop(modules, label, targetstr)
        elif mtype == "&":
            modules[label] = Conjunction(modules, label, targetstr)
            conjunctions.add(label)
        for t in targetstr.split(", "):
            pending_targets[t].append(label)
    for conjlabel in conjunctions:
        conj = modules[conjlabel]
        for inp in pending_targets[conjlabel]:
            conj.add_input(inp)
    return modules


def push_the_button(modules, times=1000):
    q = deque([])
    for _ in range(times):
        q.append(modules["broadcaster"].receive(0))
        while q:
            cb = q.popleft()
            if cb is not None:
                q.extend(cb())

In [5]:
%%time
modules = parse_puzzle(testdata)
push_the_button(modules, 1000)
sum(m.pulses[0] for m in modules.values()) * sum(m.pulses[1] for m in modules.values())

CPU times: user 7.5 ms, sys: 0 ns, total: 7.5 ms
Wall time: 7.4 ms


11687500

In [6]:
%%time
modules = parse_puzzle(data)
push_the_button(modules, 1000)
sum(m.pulses[0] for m in modules.values()) * sum(m.pulses[1] for m in modules.values())

CPU times: user 70.5 ms, sys: 317 µs, total: 70.8 ms
Wall time: 70.3 ms


886347020

### Part 2

In [7]:
%%time
modules = parse_puzzle(data)
L = []
for i in range(1, 100_000):
    q = deque({})
    gf = modules["gf"]
    q.append(modules["broadcaster"].receive(0))
    while q:
        cb = q.popleft()
        if cb is not None:
            if cb.label == "gf" and cb.pulse == 1:
                L.append((i, cb.source))
            q.extend(cb())

CPU times: user 7.89 s, sys: 52 µs, total: 7.89 s
Wall time: 7.89 s


In [8]:
len(L)

100

In [9]:
L[:12]

[(3761, 'pg'),
 (3907, 'sp'),
 (3919, 'sv'),
 (4051, 'qs'),
 (7522, 'pg'),
 (7814, 'sp'),
 (7838, 'sv'),
 (8102, 'qs'),
 (11283, 'pg'),
 (11721, 'sp'),
 (11757, 'sv'),
 (12153, 'qs')]

In [10]:
Counter(b - a for a, b in pairwise(x for (x, y) in L if y == "pg"))

Counter({3761: 25})

In [11]:
Counter(b - a for a, b in pairwise(x for (x, y) in L if y == "sp"))

Counter({3907: 24})

In [12]:
Counter(b - a for a, b in pairwise(x for (x, y) in L if y == "sv"))

Counter({3919: 24})

In [13]:
Counter(b - a for a, b in pairwise(x for (x, y) in L if y == "qs"))

Counter({4051: 23})

In [14]:
math.lcm(3761, 3907, 3919, 4051)

233283622908263